# Pré-processamento de Dados
Este notebook realiza o pré-processamento dos dados, incluindo a leitura do arquivo Excel e a criação de um dicionário de classes.

In [9]:
import os
import pandas as pd

def class_grs(grs):
    if 8 <= grs <= 15:
        return 0  # novice
    elif 16 <= grs <= 23:
        return 1  # intermediate
    elif 24 <= grs <= 31:
        return 2  # proeficient
    elif 32 <= grs <= 40:
        return 3  # specialist
    else:
        return -1  # GRS not in range

def get_classes():
    with open("data/classes.txt") as fich_labels:
        labels = fich_labels.read().split()
        classes = dict(zip(labels, list(range(len(labels)))))
    return classes

def criar_excel_limpo(videos_path, excel_path, output_excel="OSATS_clean.xlsx"):
    """
    Adiciona novos dados ao arquivo Excel 'OSATS_clean.xlsx' sem sobrescrever o conteúdo existente.
    Cada vídeo aparece apenas uma vez, com a média das classificações calculada e a classe correspondente gerada.
    
    Args:
        videos_path (str): Caminho para a pasta contendo os vídeos.
        excel_path (str): Caminho para o arquivo OSATS.xlsx.
        output_excel (str): Nome do arquivo Excel de saída.
    """
    # Verificar se os caminhos existem
    if not os.path.exists(videos_path):
        raise FileNotFoundError(f"Pasta de vídeos não encontrada: {videos_path}")
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Arquivo Excel não encontrado: {excel_path}")
    
    # Carregar o arquivo Excel original
    df = pd.read_excel(excel_path)
    
    # Listar todos os arquivos de vídeo na pasta
    video_extensions = ('.mp4',)
    videos = [os.path.splitext(f)[0] for f in os.listdir(videos_path) if f.lower().endswith(video_extensions)]
    
    # Filtrar o DataFrame para conter apenas os vídeos presentes na pasta
    df_filtered = df[df['VIDEO'].isin(videos)]
    
    # Calcular a média do GLOBA_RATING_SCORE para cada vídeo
    df_mean = df_filtered.groupby('VIDEO')['GLOBA_RATING_SCORE'].mean().reset_index()
    
    # Adicionar a coluna 'GRS' com a classificação baseada na média
    df_mean['GRS'] = df_mean['GLOBA_RATING_SCORE'].apply(class_grs)
    
    # Selecionar apenas as colunas 'VIDEO' e 'GRS'
    df_clean = df_mean[['VIDEO', 'GRS']]
    
    # Verificar se o arquivo de saída já existe
    if os.path.exists(output_excel):
        # Carregar o conteúdo existente
        existing_df = pd.read_excel(output_excel)
        
        # Concatenar os novos dados com os existentes, evitando duplicatas
        combined_df = pd.concat([existing_df, df_clean]).drop_duplicates(subset=['VIDEO']).reset_index(drop=True)
    else:
        # Caso o arquivo não exista, usar apenas os novos dados
        combined_df = df_clean
    
    # Salvar o arquivo atualizado
    combined_df.to_excel(output_excel, index=False)
    print(f"Arquivo '{output_excel}' atualizado com sucesso!")

In [12]:
# Exemplo de uso
VIDEO_PATH = "data/Package02"
EXCEL_PATH = "data/OSATS.xlsx"
criar_excel_limpo(VIDEO_PATH, EXCEL_PATH)
clean_excel_file = pd.read_excel("OSATS_clean.xlsx")
clean_excel_file

Arquivo 'OSATS_clean.xlsx' atualizado com sucesso!


,VIDEO,GRS
0,A31H,0
1,A40E,1
2,A41X,3
3,A61V,2
4,A80U,0
5,A83X,2
6,A92O,1
7,B12G,0
8,B61D,0
9,C16G,1
